In [58]:
import re
import pandas as pd
from sqlalchemy import create_engine

In [8]:
chess_data = open("chess.pgn").read()

In [35]:
#regex
date_regex = "(?<=\[Date \")(.*)(?=\"])"
white_regex = "(?<=\[White \")(.*)(?=\"])"
black_regex = "(?<=\[Black \")(.*)(?=\"])"
result_score_regex = "(?<=\[Result \")(.*)(?=\"])"
match_time_regex = "(?<=\[TimeControl \")(.*)(?=\"])"
end_result = "(?<=\[Termination \")(.*)(?=\"])"

In [36]:
dates = re.findall(date_regex, chess_data)
white_player = re.findall(white_regex, chess_data)
black_player = re.findall(black_regex, chess_data)
result_score = re.findall(result_score_regex, chess_data)
match_time_and_inc = re.findall(match_time_regex, chess_data)
end_result = re.findall(end_result, chess_data)

if_white_won = list(map(lambda s: s[0], result_score))
if_black_won = list(map(lambda s: s[2], result_score))

In [40]:
start_time_amount = list()
increments = list()
for time in match_time_and_inc:
    if "+" in time:
        start_time_amount.append(time[:-3])
        increments.append(time[-2:])
    else:
        start_time_amount.append(time)
        increments.append(None)
    

In [47]:
game_ended_by = list()

for end_string in end_result:
    if "time" in end_string:
        game_ended_by.append("Time")
    elif "checkmate" in end_string:
        game_ended_by.append("Checkmate")
    elif "game abandoned" in end_string:
        game_ended_by.append("Abandoned")
    elif "drawn by agreement" in end_string:
        game_ended_by.append("Drawn by Aggreement")
    elif "stalemate" in end_string:
        game_ended_by.append("Stalemate")
    elif "resignation" in end_string:
        game_ended_by.append("Resignation")
    else:
        game_ended_by.append(None)

In [49]:

start_time_amount_minutes = list(map(lambda n: int(n)//60, start_time_amount))
print(start_time_amount_minutes)

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 10, 10, 10, 10, 10, 10, 10, 10, 15, 10, 10, 5, 10, 10, 10, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 5, 10, 10, 10, 10, 10, 20, 10, 15, 1, 15]


In [59]:
#dates, white_player, black_player, if_white_won, if_black_won, start_time_amount, increments, start_time_amount_minutes
chess_final_data = pd.DataFrame(
    {
        'datePlayed': dates,
        'whitePlayer': white_player,
        'blackPlayer': black_player,
        'ifWhiteWon': if_white_won,
        'ifBlackWon': if_black_won,
        'startTimeSeconds': start_time_amount,
        'secondIncrements': increments,
        'startTimeMinutes': start_time_amount_minutes 
    }
)

In [68]:
chess_final_data[['ifWhiteWon', 'ifBlackWon', 'startTimeSeconds', 'secondIncrements', 'startTimeMinutes']] = chess_final_data[['ifWhiteWon', 'ifBlackWon', 'startTimeSeconds', 'secondIncrements', 'startTimeMinutes']].apply(pd.to_numeric)
chess_final_data['datePlayed'] = pd.to_datetime(chess_final_data['datePlayed'])
chess_final_data

#CREATE TABLE CHESS (D)

,datePlayed,whitePlayer,blackPlayer,ifWhiteWon,ifBlackWon,startTimeSeconds,secondIncrements,startTimeMinutes
0,2023-07-19,Nathan-Hung,squishthatcat,1,0,300,NaN,5
1,2023-07-19,Nathan-Hung,squishthatcat,1,0,300,NaN,5
2,2023-07-19,Nathan-Hung,squishthatcat,1,0,300,NaN,5
3,2023-07-19,squishthatcat,Nathan-Hung,0,1,300,NaN,5
4,2023-07-19,Nathan-Hung,squishthatcat,1,0,300,NaN,5
5,2023-07-19,squishthatcat,Nathan-Hung,1,0,300,NaN,5
6,2023-07-19,Nathan-Hung,squishthatcat,1,0,300,NaN,5
7,2023-07-19,Nathan-Hung,squishthatcat,1,0,300,NaN,5
8,2023-07-19,squishthatcat,Nathan-Hung,1,0,300,NaN,5
9,2023-07-19,Nathan-Hung,squishthatcat,1,0,300,NaN,5
